In [1]:
from litellm import completion
from dotenv import load_dotenv
import json
from pricer.batch import Batch
from pricer.items import Item

load_dotenv(override=True)

True

In [77]:
import sys
!{sys.executable} -m pip install ipywidgets jupyterlab_widgets

In [4]:
username = "kshitijchaudhary"
dataset = f"{username}/items_raw_lite"

train, val, test = Item.from_hub(dataset)

items = train + val + test

print(f"Loaded {len(items):,} items")
print(items[0])


Generating test split: 100%|██████████| 1000/1000 [00:00<00:00, 237462.72 examples/s]


Loaded 22,000 items
title='Odyssey K45120BLK Krom Utility/Record Case, Black' category='Musical_Instruments' price=119.95 full='Odyssey  Krom Utility/Record Case, Black\n[\'Protect your rare and hard to find records used for your special 45 rpm vinyl sets or just for safe keeping in our Krom™ series  black utility/record cases. Each has two individual compartments that hold a total of 120 7" vinyl records, 60 in each section, and can also be used as a utility case for cables and other essentials. Features include chrome plated hardware, a removable lid, a fully foam-lined interior and a heavy-duty latch and spring-loaded handle. Individual K45120 cases can be stacked on each other for convenient storage. Also available in silver ||.\']\n[\'Butterfly latch and spring-loaded handle\', \'Padlock latch holes\', \'Holds 120 7" vinyl records, 60 per compartment\', \'Foam-lined interior\', \'Rubber feet and stacking lid\']\n{"Item Weight": "7 pounds", "Product Dimensions": "9.25 x 17.75 x 9.7

In [11]:
items[0].id

0

In [9]:
for index, item in enumerate(items):
    item.id = index


In [13]:
SYSTEM_PROMPT = """Create a concise description of a product. Respond only in this format. Do not include part numbers.
Title: Rewritten short precise title
Category: eg Electronics
Brand: Brand name
Description: 1 sentence description
Details: 1 sentence on features"""

In [18]:
print(items[1].full)

HP 15t-dw300 FHD IPS Laptop (11th Gen Intel i5 4-Core, 16GB RAM, 512GB PCIe SSD, Intel Iris Xe, 15.6" WLED (1920x1080), WiFi, Bluetooth, Webcam, 2xUSB 3.1, 1xHDMI, Win 10 Home) with Hub
['GreatPriceTech sells computers with custom/upgraded configurations to enhance system performance. If the computer has modifications as listed above, the manufacturer’s box was opened by our highly skilled technicians for testing, inspection, and installation of the upgrades according to the specifications advertised. All computers and components are brand new.', 'Processor: Intel Core i5 2.40GHz Processor (11th Gen, upto 4.2 GHz, 8MB Cache, 4-Cores)', 'Processor:', 'Intel Core i5 2.40GHz Processor (11th Gen, upto 4.2 GHz, 8MB Cache, 4-Cores)', 'Storage: 512GB PCIe SSD (Solid State Drive)', 'Storage:', '512GB PCIe SSD (Solid State Drive)', 'Memory: 16GB DDR4 SO-DIMM', 'Memory:', '16GB DDR4 SO-DIMM', 'Graphics: Intel Iris Xe Integrated Graphics,', 'Graphics:', 'Intel Iris Xe Integrated Graphics,', 'Oper

In [ ]:
messages = [{"role": "system", "content": SYSTEM_PROMPT},{"role": "user", "content": items[1].full}]

response = completion(messages=messages, model="groq/openai/gpt-oss-20b", reasoning_effort="low")

response = completion(messages=messages, model="groq/openai/gpt-oss-20b", reasoning_effort="low")
print(response.choices[0].message.content)

Title: HP 15t‑dw300 15.6" FHD IPS Laptop  
Category: Electronics  
Brand: HP  
Description: 15.6" Full HD IPS laptop with 11th‑Gen Intel i5, 16 GB RAM, 512 GB SSD and Intel Iris Xe graphics.  
Details: Equipped with Wi‑Fi, Bluetooth, webcam, dual USB 3.1, HDMI, and a USB hub accessory.


In [20]:
MODEL = "openai/gpt-oss-20b"

In [52]:
def make_jsonl(item):
    body = {"model":MODEL, "messages":[{"role": "system", "content": SYSTEM_PROMPT},{"role": "user", "content": item.full}], "reasoning_effor":"low"}
    line = {"custom_id":str(item.id), "method":"POST", "url":"/v1/chat/completions", "body":body}
    return json.dumps(line)

In [53]:
make_jsonl(items[1])

'{"custom_id": "1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "openai/gpt-oss-20b", "messages": [{"role": "system", "content": "Create a concise description of a product. Respond only in this format. Do not include part numbers.\\nTitle: Rewritten short precise title\\nCategory: eg Electronics\\nBrand: Brand name\\nDescription: 1 sentence description\\nDetails: 1 sentence on features"}, {"role": "user", "content": "HP 15t-dw300 FHD IPS Laptop (11th Gen Intel i5 4-Core, 16GB RAM, 512GB PCIe SSD, Intel Iris Xe, 15.6\\" WLED (1920x1080), WiFi, Bluetooth, Webcam, 2xUSB 3.1, 1xHDMI, Win 10 Home) with Hub\\n[\'GreatPriceTech sells computers with custom/upgraded configurations to enhance system performance. If the computer has modifications as listed above, the manufacturer\\u2019s box was opened by our highly skilled technicians for testing, inspection, and installation of the upgrades according to the specifications advertised. All computers and components are brand

In [55]:
def make_file(start, end, filename):
    batch_file = filename
    with open(batch_file, 'w') as f:
        for i in range(start, end):
            f.write(make_jsonl(items[i]))
            f.write("\n")

In [56]:
make_file(0, 1000, "jsonl/0_1000.jsonl")

## Start creating Batch file for Groq

In [57]:
import os
from groq import Groq

groq = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [ ]:
import os
from groq import Groq

groq = Groq(api_key=os.environ.get("GROQ_API_KEY"))

LITE_MODE = True

username = "ed-donner"
dataset = f"{username}/items_raw_lite" if LITE_MODE else f"{username}/items_raw_full"

train, val, test = Item.from_hub(dataset)

items = train + val + test

print(f"Loaded {len(items):,} items")

for index, item in enumerate(items):
    item.id = index

SYSTEM_PROMPT = """Create a concise description of a product. Respond only in this format. Do not include part numbers.
Title: Rewritten short precise title
Category: eg Electronics
Brand: Brand name
Description: 1 sentence description
Details: 1 sentence on features"""

MODEL = "openai/gpt-oss-20b"

def make_jsonl(item):
    body = {"model": MODEL, "messages": [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": item.full}], "reasoning_effort": "low"}
    line = {"custom_id": str(item.id), "method": "POST", "url": "/v1/chat/completions", "body": body}
    return json.dumps(line)

def make_file(start, end, filename):
    batch_file = filename
    with open(batch_file, "w") as f:
        for i in range(start, end):
            f.write(make_jsonl(items[i]))
            f.write("\n")

make_file(0, 1000, "jsonl/0_1000.jsonl")

with open("jsonl/0_1000.jsonl", "rb") as f:
    response = groq.files.create(file=f, purpose="batch")

file_id = response.id
response = groq.batches.create(completion_window="24h", endpoint="/v1/chat/completions", input_file_id=file_id)
result = groq.batches.retrieve(response.id)
response = groq.files.content(result.output_file_id)
response.write_to_file("jsonl/batch_results.jsonl")


Loaded 22,000 items


ValueError: Expected a non-empty value for `file_id` but received None

In [4]:
result = groq.batches.retrieve(response.id)
response = groq.files.content(result.output_file_id)
response.write_to_file("jsonl/batch_results.jsonl")

In [5]:
Batch.create(items, LITE_MODE)

Created 22 batches


In [6]:
Batch.run()

  0%|          | 0/22 [00:00<?, ?it/s]

Submitted 22 batches


In [7]:
Batch.fetch()

  0%|          | 0/22 [00:00<?, ?it/s]

Finished 22 of 22 batches


In [9]:
for index, item in enumerate(items):
    if not item.summary:
        print(index)

In [12]:
print(items[3254].summary)

Title: Foldable Aluminum Tablet Stand  
Category: Electronics  
Brand: Unknown  
Description: A slim, lightweight stand that supports tablets from 7" to 13" in any orientation.  
Details: Made of sturdy aluminum alloy with silicone grip, adjustable across 8 angles, and folds flat for easy storage.


In [13]:
username='kshitijchaudhary'
lite = f"{username}/items_lite"

train = items[:20_000]
val = items[20_000:21_000]
test = items[21_000:]

Item.push_to_hub(lite, train, val, test)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

...

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            